In [1]:
import sys

sys.path.append('../../')

In [2]:
from finance.discountfactor import ConstantRateDiscountFactor 

r = 0.02
discount = ConstantRateDiscountFactor(r)

In [3]:
import numpy as np
from maths.montecarlo.processes.brownianmotions import BrownianMotion

maturity = 5.
step = 1./360
time_grid = BrownianMotion.generate_time_grid(0, maturity, step)

x_0 = [100, 100]
mu_s = [[5], [4]]
sigma_s = [[10], [6]]

correl = [[1, 0], [0, 1]]

b = BrownianMotion(time_grid, x_0, mu_s, sigma_s, correl)
print b.values[:, 0]
print b(0)

[ 100.  100.]
[[ 100.]
 [ 100.]]


In [4]:
from finance.products.european.assetswap import (
    SwapContract,
)

delta = 0.25
coupon_dates = SwapContract.generate_payment_dates(0, maturity, delta)

swap = SwapContract(b, discount, coupon_dates)
print swap

Swap contract of maturity T = 5 years, over S^0 with strike K = 104.821, paying at {0.00, 0.25, 0.50, 0.75, 1.00, 1.25, 1.50, 1.75, 2.00, 2.25, 2.50, 2.75, 3.00, 3.25, 3.50, 3.75, 4.00, 4.25, 4.50, 4.75, 5.00}


In [5]:
from risk.exposures import EuropeanQuantileBrownianExposure

exposure_mpor = 1./360
exposure_quantile = 0.99

exposure = EuropeanQuantileBrownianExposure(swap, discount, mu_s[0], sigma_s[0])

In [6]:
from finance.portfolio import CCPPortfolio

pos = [[1],
       [-2],
       [1]]

port = CCPPortfolio(pos, [swap], [exposure])

In [7]:
from ccp.states import MembersState

states = MembersState(3)
states.alive_states

array([ True,  True,  True], dtype=bool)

In [8]:
from ccp.accounts import Accounts, DFAccounts

vm_accounts = Accounts(states, 1)
im_accounts = Accounts(states, 1)
df_accounts = DFAccounts(states, 1)

df_accounts.put_amounts(1, 1000)
df_accounts.put_amounts(0, 50)
states.die(2)

print df_accounts.amounts

[[   50.]
 [ 1000.]
 [    0.]]


In [9]:
from ccp.sig import SkinInTheGame

sig = SkinInTheGame(0.)

In [10]:
from risk.basel import CCPRegulatoryCapital

beta = 0.3

regul_cap = CCPRegulatoryCapital(vm_accounts, im_accounts, df_accounts, sig, beta, port)

In [11]:
regul_cap.compute_kcm(1, 0)

6.2545847941076422